----------------------------------------------------
Modelagem de tópicos de documentos provenientes da base de dados do CPDOC.
----------------------------------------------------
Base de dados: Antonio Azeredo da Silveira.
-----------------------------
Número de documentos: 10.428
-----------------------------
Número de imagens: 66.938
-----------------------------

### Topic: Clustering documents

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyLDAvis
import pyLDAvis.gensim

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead

Specifying the path to the files  

In [2]:
#datapath = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected' # path marcelo
datapath = "../textfiles-corrected"
outputs = "../outputs/"
import os

onlyfiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath,f))]

In [3]:
onlyfiles[0:4]

['txt_0007168_484.txt',
 'txt_0062976_89.txt',
 'txt_0023552_242.txt',
 'txt_0002560_270.txt']

#### make list of txt contents of files

In [4]:
count = 0
percentil = int(len(onlyfiles)/100)
textos = []

for txt in onlyfiles:
    with open(os.path.join(datapath,txt), 'r', encoding='utf-8') as f:
        count += 1
        if count % percentil == 0: print(int(count/percentil),'% done')
        texto = f.read()
        texto = texto.split()
        #print(len(texto))
        symbols = [x for x in string.punctuation]
        texto = [p for p in texto if p not in symbols]
        #print(len(texto))
        texto = [p.strip(string.punctuation) for p in texto]
        #print(len(texto))
        texto = [p for p in texto if not p.isdigit()]
        #print(len(texto))
        texto = [p for p in texto if len(p)>1]
        #print(len(texto))
        #print(texto)
        textos.append(texto)
#print(textos)

1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done


### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the synopses. First, I load [NLTK's](http://www.nltk.org/) list of English stop words. [Stop words](http://en.wikipedia.org/wiki/Stop_words) are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [5]:
# load nltk's English stopwords as variable called 'stopwords'
additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
            
stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [6]:
#Latent Dirichlet Allocation implementation with Gensim
from gensim import corpora, models, similarities 

%time texts = [[word for word in text if word not in stopwords] for text in textos]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/configparser.py:81: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  'PID': str(os.getpid()),
Using gpu device 0: Quadro K4200 (CNMeM is disabled, cuDNN 5103)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


CPU times: user 2min 2s, sys: 0 ns, total: 2min 2s
Wall time: 2min 2s


In [7]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

995


In [8]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [9]:
len(corpus)

66908

In [10]:
%time lda = models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=10)

CPU times: user 37min 17s, sys: 10min 49s, total: 48min 7s
Wall time: 35min 54s


In [11]:
lda.print_topics(20, num_words=20)

[(0,
  '0.012*política + 0.011*desenvolvimento + 0.009*brasil + 0.009*internacional + 0.008*relações + 0.006*mundo + 0.005*nações + 0.005*cooperação + 0.005*interesses + 0.005*econômica + 0.004*américa + 0.004*internacionais + 0.004*econômico + 0.004*cada + 0.004*problemas + 0.004*plano + 0.004*externa + 0.003*mundial + 0.003*latina + 0.003*processo'),
 (1,
  '0.018*governo + 0.009*brasil + 0.007*presidente + 0.006*ter + 0.005*brasileiro + 0.005*sido + 0.004*posição + 0.004*quanto + 0.004*disse + 0.004*carter + 0.004*política + 0.004*fato + 0.004*situação + 0.004*questão + 0.003*pois + 0.003*agora + 0.003*eua + 0.003*fazer + 0.003*vez + 0.003*segundo'),
 (2,
  '0.008*total + 0.006*etc + 0.006*itaipú + 0.006*valor + 0.005*cuenca + 0.005*tabela + 0.005*plata + 0.005*potencia + 0.005*valores + 0.004*base + 0.004*grupos + 0.004*central + 0.004*intereses + 0.004*articulo + 0.003*integración + 0.003*antártida + 0.003*fonte + 0.003*100,0 + 0.003*formas + 0.002*tramo'),
 (3,
  '0.027*republic 

In [12]:
lda.show_topics()

[(2,
  '0.008*total + 0.006*etc + 0.006*itaipú + 0.006*valor + 0.005*cuenca + 0.005*tabela + 0.005*plata + 0.005*potencia + 0.005*valores + 0.004*base'),
 (9,
  '0.029*relações + 0.026*brasil + 0.020*excelência + 0.018*ministro + 0.018*presidente + 0.017*senhor + 0.017*exteriores + 0.016*governo + 0.012*república + 0.010*visita'),
 (5,
  '0.010*brasil + 0.009*gobierno + 0.006*paraguay + 0.006*relaciones + 0.005*si + 0.005*acuerdo + 0.005*argentina + 0.005*general + 0.004*artículo + 0.004*reunion'),
 (15,
  '0.007*mw + 0.006*ar + 0.005*cl + 0.004*lu + 0.004*rr + 0.004*cn + 0.004*nn + 0.004*ãe + 0.004*uu + 0.004*ln'),
 (6,
  '0.018*brasil + 0.009*cooperação + 0.007*produtos + 0.007*us + 0.006*energia + 0.006*comércio + 0.006*comercial + 0.005*produção + 0.005*petróleo + 0.005*desenvolvimento'),
 (14,
  '0.013*artigo + 0.010*acordo + 0.007*art + 0.007*lei + 0.005*nacional + 0.005*tratado + 0.005*federal + 0.004*presente + 0.004*ministério + 0.004*cada'),
 (17,
  '0.008*world + 0.008*state

In [13]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

In [14]:
topics_matrix[3]

(15,
 [('mw', 0.0065063168778169023),
  ('ar', 0.0060867530762961173),
  ('cl', 0.0049703400443155124),
  ('lu', 0.0044875937132731738),
  ('rr', 0.0042321608846394455),
  ('cn', 0.004137249130664121),
  ('nn', 0.0040574604596183691),
  ('ãe', 0.0040305961549562848),
  ('uu', 0.0039373236802308738),
  ('ln', 0.0038792369754158357),
  ('rn', 0.0038102485400075968),
  ('ae', 0.0037589508874920504),
  ('dn', 0.0036642494502752703),
  ('ue', 0.0036293035308740508),
  ('mu', 0.003467574306245494),
  ('ão', 0.0034371188882854555),
  ('ml', 0.0032662642313953591),
  ('cc', 0.0032022360270881067),
  ('io', 0.0031977072562770197),
  ('www', 0.00319371431438947)])

In [15]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      9.924420        1       1  0.220485  0.129502
1      9.805321        1       2  0.194953  0.031888
9      9.105472        1       3  0.171463 -0.102034
13     8.033452        1       4 -0.287295  0.228542
12     8.033048        1       5  0.241628  0.107309
6      8.005873        1       6  0.186117  0.096649
14     6.839803        1       7  0.185555  0.051779
17     6.012457        1       8 -0.283360  0.203151
19     5.385793        1       9  0.072974 -0.119821
16     5.043159        1      10 -0.013722 -0.221393
11     3.710097        1      11  0.130197  0.048321
10     3.550586        1      12 -0.050613 -0.238162
15     3.149676        1      13 -0.135090 -0.198579
18     3.049785        1      14  0.060934  0.031358
5      2.940957        1      15  0.064962  0.049012
7      1.868036        1      16 -0.060865 -0.071252
3      1.579970        1      17 -0.190057  0.037969
4      1.441617        1      18 -0.199094  0.033152
8      1.374190        1      19 -0.185581 -0.085618
2      1.146289        1      20 -0.123592 -0.011772, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
73495  Default  19804.000000           senhor  19804.000000  30.0000  30.0000
94728  Default  21156.000000         ministro  21156.000000  29.0000  29.0000
93658  Default  13035.000000        telegrama  13035.000000  28.0000  28.0000
29671  Default  24099.000000         relações  24099.000000  27.0000  27.0000
25914  Default  50394.000000           brasil  50394.000000  26.0000  26.0000
28181  Default  23745.000000          governo  23745.000000  25.0000  25.0000
33755  Default  21519.000000       presidente  21519.000000  24.0000  24.0000
47402  Default  13077.000000       excelência  13077.000000  23.0000  23.0000
29201  Default  12399.000000       embaixador  12399.000000  22.0000  22.0000
31653  Default  11378.000000           united  11378.000000  21.0000  21.0000
30237  Default   8737.000000         silveira   8737.000000  20.0000  20.0000
87447  Default   9204.000000        argentina   9204.000000  19.0000  19.0000
34593  Default   6632.000000           itaipu   6632.000000  18.0000  18.0000
96090  Default  10766.000000              rio  10766.000000  17.0000  17.0000
63042  Default  16882.000000       exteriores  16882.000000  16.0000  16.0000
47154  Default   7821.000000          nations   7821.000000  15.0000  15.0000
29362  Default   7652.000000         comissão   7652.000000  14.0000  14.0000
8338   Default   9537.000000            geral   9537.000000  13.0000  13.0000
67626  Default  12579.000000        república  12579.000000  12.0000  12.0000
7490   Default   7142.000000         recebido   7142.000000  11.0000  11.0000
63937  Default   8530.000000        chanceler   8530.000000  10.0000  10.0000
85773  Default   6707.000000           nações   6707.000000   9.0000   9.0000
84288  Default   6015.000000          azeredo   6015.000000   8.0000   8.0000
53663  Default   6836.000000        committee   6836.000000   7.0000   7.0000
61059  Default   7897.000000    international   7897.000000   6.0000   6.0000
42319  Default  11622.000000  desenvolvimento  11622.000000   5.0000   5.0000
55806  Default   7713.000000          projeto   7713.000000   4.0000   4.0000
89652  Default   4903.000000          antonio   4903.000000   3.0000   3.0000
58959  Default   4750.000000           corpus   4750.000000   2.0000   2.0000
4211   Default   9096.000000           visita   9096.000000   1.0000   1.0000
...        ...           ...              ...           ...      ...      ...
96310  Topic20    121.958314       propuestas    122.895038   4.4610  -6.4229
1749   Topic20    120.807316           iguazu    121.744040   4.4609  -6.4324
25967  Topic20    120.475407              0,5    121.412132  

In [16]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))

In [17]:
%time hdp = models.HdpModel(corpus, id2word=dictionary)

CPU times: user 38min 37s, sys: 2h 45min 27s, total: 3h 24min 4s
Wall time: 16min 1s


In [18]:
hdp.print_topics()

['topic 0: 0.010*brasil + 0.005*governo + 0.005*relações + 0.004*presidente + 0.003*ministro + 0.003*senhor + 0.003*exteriores + 0.003*política + 0.003*acordo + 0.003*brasileiro + 0.002*desenvolvimento + 0.002*cooperação + 0.002*excelência + 0.002*república + 0.002*internacional + 0.002*brasileira + 0.002*telegrama + 0.002*unidos + 0.002*geral + 0.002*embaixador',
 'topic 1: 0.010*united + 0.007*states + 0.007*international + 0.007*nations + 0.006*committee + 0.006*general + 0.004*assembly + 0.004*economic + 0.004*resolution + 0.004*development + 0.004*world + 0.003*report + 0.003*brazil + 0.003*nuclear + 0.003*government + 0.003*session + 0.003*state + 0.002*meeting + 0.002*developing + 0.002*paragraph',
 'topic 2: 0.009*brasil + 0.007*ministro + 0.007*senhor + 0.005*presidente + 0.005*telegrama + 0.005*relações + 0.005*exteriores + 0.004*governo + 0.004*embaixador + 0.004*excelência + 0.003*chanceler + 0.003*república + 0.003*silveira + 0.002*recebido + 0.002*secreto + 0.002*visita +

In [19]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(hdp, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      53.821323        1       1  0.330939  0.047081
1      15.231920        1       2  0.107133 -0.452438
2      11.477022        1       3  0.341193  0.041266
3       5.794152        1       4  0.316991  0.033103
4       4.029111        1       5  0.327205  0.028191
6       3.819341        1       6  0.309111  0.011173
5       1.408009        1       7  0.063527 -0.200956
8       0.787215        1       8  0.023032 -0.172831
7       0.502644        1       9  0.232310  0.024375
9       0.391987        1      10  0.155234  0.041096
10      0.239391        1      11  0.149416  0.028458
14      0.165956        1      12  0.098330  0.031391
15      0.152593        1      13  0.087920  0.020696
13      0.151012        1      14  0.113938  0.036216
23      0.143295        1      15  0.020591  0.015374
12      0.139368        1      16  0.091664 -0.000404
11      0.129761        1      17 -0.005323 -0.194228
22      0.111774        1      18  0.016132 -0.005013
16      0.094069        1      19  0.067287  0.002758
21      0.069925        1      20  0.036894  0.018801
19      0.065986        1      21  0.063580  0.025447
17      0.061661        1      22  0.059597  0.003960
29      0.055897        1      23  0.018816  0.018318
46      0.047386        1      24 -0.034839  0.012391
54      0.044878        1      25 -0.012246  0.019474
25      0.043260        1      26  0.026902  0.008609
53      0.043060        1      27 -0.008624  0.016243
33      0.042491        1      28  0.003114  0.014655
39      0.037013        1      29 -0.011101 -0.004637
59      0.035509        1      30 -0.019218  0.010561
...          ...      ...     ...       ...       ...
91      0.001895        1     121 -0.031990  0.005181
82      0.001794        1     122 -0.025429  0.001858
136     0.001764        1     123 -0.038355  0.013444
137     0.001651        1     124 -0.035892  0.012868
114     0.001614        1     125 -0.030742  0.012078
111     0.001528        1     126 -0.031504  0.006488
115     0.001515        1     127 -0.035503  0.005715
108     0.001404        1     128 -0.035336  0.007204
143     0.001394        1     129 -0.040050  0.010002
121     0.001385        1     130 -0.034173  0.014162
97      0.001370        1     131 -0.029815  0.014694
140     0.001219        1     132 -0.032377  0.013275
123     0.001154        1     133 -0.034110  0.014271
103     0.001136        1     134 -0.030323  0.004214
122     0.000993        1     135 -0.034683  0.010220
142     0.000946        1     136 -0.041499  0.006608
106     0.000885        1     137 -0.029618  0.005581
144     0.000873        1     138 -0.045749  0.008086
127     0.000858        1     139 -0.039133  0.006515
131     0.000854        1     140 -0.037163  0.013278
85      0.000838        1     141 -0.040516 -0.006990
146     0.000822        1     142 -0.043213  0.011356
130     0.000815        1     143 -0.033610  0.013763
126     0.000630        1     144 -0.036007  0.007913
149     0.000559        1     145 -0.040658  0.011805
141     0.000534        1     146 -0.041793  0.011868
128     0.000298        1     147 -0.035823  0.010678
147     0.000278        1     148 -0.041710  0.011176
145     0.000157        1     149 -0.042909  0.011489
148     0.000133        1     150 -0.044362  0.012856

[150 rows x 5 columns], topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
25914   Default  48235.000000           brasil  48235.000000  30.0000  30.0000
94728   Default  24504.000000         ministro  24504.000000  29.0000  29.0000
73495   Default  23067.000000           senhor  23067.000000  28.0000  28.0000
29671   Default  24039.000000         relações  24039.000000  27.0000  27.0000
33755   Default  22973.000000       presidente  22973.000

In [20]:
data_hdpvis = pyLDAvis.gensim.prepare(hdp, corpus, dictionary)
pyLDAvis.save_html(data_hdpvis, os.path.join(outputs,'pyldavis_output_hdp.html'))